## Earth Observation (EO) Data Preparation

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import num2date, Dataset
import xarray as xr
from tools import CreateGridBox, CreateGridBox_subextent, save_hdf, IntersectShapefiles
import rasterio
import xarray as xr

### First checking the administrative boundaries
Here we add admin level-1 FNID to the admin level-2 shapefile.

In [2]:
# Load FEWSNET admin boundaries of Somalia (1990)
adm1_1990 = gpd.read_file('./data/shapefile/SO_Admin1_1990.shp')   # Total 18 FNID
adm2_1990 = gpd.read_file('./data/shapefile/SO_Admin2_1990.shp')   # Total 74 FNID
# - Insert admin_1 FNID to the admin_2 shapefile
adm1_1990 = adm1_1990.rename({'FNID':'FNID_ADM1'}, axis=1)
adm2_1990 = adm2_1990.merge(adm1_1990[['FNID_ADM1','ADMIN1']], on='ADMIN1')
adm2_1990 = adm2_1990[np.roll(adm2_1990.columns, 1)]
adm2_1990.to_file('./data/shapefile/SO_admin2_1990_revised.shp')
# Get sub_extent from shapefile
sub_bound = adm2_1990.total_bounds   # [left, bottom, right, top] 
sub_extent = sub_bound[[0,2,1,3]]    # [minx,maxx,miny,maxy]

### 1) CHIRPS Precipitation
CHIRPS data: /home/ftp_out/products/CHIRPS-2.0/global_monthly/netcdf/

We create a graticule of the CHIRPS-Africa for the extent of the country, then we intersect it with admin level-2 shapefile and calculate an area of each segment (grid or partial grid).

In [24]:
# Get the extent of CHIRPS data
with rasterio.open('./data/earthobs/CHIRPS/chirps-v2.0.2020.01.01.tif') as src:
    affine = src.meta['transform']
    dx, dy = affine.a, -affine.e
    bounds = np.array(src.bounds)   # [left, bottom, right, top] 
    extent = bounds[[0,2,1,3]]      # [minx,maxx,miny,maxy]

# (1) Create a graticule of CHIRPS data
filn_out = './data/shapefile/SO_admin2_chirps.shp'
CreateGridBox_subextent(filn_out, extent, dx, dy, sub_extent, set_print=False)
    
# (2) Intersection of CHIRPS with admin level-2 boundaries
ref = './data/shapefile/SO_admin2_chirps.shp'
adm = './data/shapefile/SO_admin2_1990_revised.shp'
IntersectShapefiles(ref, adm, outSHP=ref, set_print=False)

# (3) Calculate the areas of the intersected polygons
# Reproject to Somalia UTM: UTM zone 38N (EPSG:20538)
filn_out = './data/shapefile/SO_admin2_chirps.shp'
grid_dist = gpd.read_file('./data/shapefile/SO_admin2_chirps.shp')
grid_dist = grid_dist.to_crs(epsg=20538)    # UTM zone 18S
# Caculate areas of all split polygons
grid_dist["area_km2"] = grid_dist['geometry'].area / 10**6
# Save GeoDataFrame to Shapefile
grid_dist.to_file(filn_out)
print('%s is saved.' % filn_out)

./data/shapefile/SO_admin2_chirps.shp is saved.


In [25]:
# Load CHIPRS data
temp = np.load('/Users/dlee/data/chirps/africa_monthly_198101_202008.npz', allow_pickle=True)
prcp = temp['data']
ntim, nlat, nlon = prcp.shape
lat, lon, tim = temp['lat'], temp['lon'], temp['tim']
tim = [str(period) for period in tim]
tim = pd.to_datetime(tim, format='%Y-%m')
tim = tim.to_period('M')
prcp = prcp.reshape([ntim,nlat*nlon])

# Spatial averages in administrative units
grid = gpd.read_file('./data/shapefile/SO_admin2_chirps.shp')
listID = np.sort(grid.FNID.unique())
nID = len(listID)
prcp_dist = pd.DataFrame(np.zeros([ntim,nID]), index=tim, columns=listID)
prcp_dist.index.name = 'time'
for fnid in listID:
    subID = grid[grid.FNID == fnid].ID.astype(int)
    subData = prcp[:, subID.values]
    subData[np.isnan(subData)] = 0    # Assume 0 values at NaN grids
    subArea = grid[grid.FNID == fnid].area_km2
    # Area-averaged data
    prcp_dist[fnid] = (subData @ subArea)/subArea.sum()
save_hdf('./data/earthobs/SO_admin2_prcp.hdf',prcp_dist)
del prcp, lat, lon, tim
prcp_dist.head()

./data/SO_admin2_prcp.hdf is saved.


,SO1990A21101,SO1990A21102,SO1990A21103,SO1990A21104,SO1990A21201,SO1990A21202,SO1990A21203,SO1990A21301,SO1990A21302,SO1990A21303,...,SO1990A22604,SO1990A22605,SO1990A22606,SO1990A22701,SO1990A22702,SO1990A22703,SO1990A22801,SO1990A22802,SO1990A22803,SO1990A22804
time,,,,,,,,,,,,,,,,,,,,,
1981-01,2.912159,3.393857,3.959673,4.684031,1.688224,3.054894,1.573421,1.683044,0.886342,1.970712,...,0.935361,0.666849,0.587090,0.503367,0.520322,0.666045,0.908092,1.167884,1.697125,0.460263
1981-02,6.108098,3.403615,3.825072,5.734320,3.422284,0.816510,4.052085,2.131610,1.645105,3.141231,...,3.449212,2.025924,1.225686,1.895644,1.436128,2.996263,2.233642,4.323137,3.012077,1.668710
1981-03,98.124783,37.571041,24.489698,38.142071,69.228262,5.022367,80.427226,19.767632,12.826464,56.478382,...,175.486449,80.770001,83.535444,56.259073,23.186481,88.210897,67.812608,87.942516,90.648058,21.433080
1981-04,54.722570,41.421288,17.285735,13.692762,54.295913,54.075556,60.011767,48.734963,46.363409,42.619214,...,178.953473,159.615907,237.558113,260.007454,212.241230,243.490429,128.639451,222.602866,124.473864,148.601705
1981-05,20.387662,15.846781,5.546454,5.490030,29.849543,36.360596,28.079327,34.120321,25.020623,33.296629,...,57.342453,49.820345,50.529063,140.364868,200.955812,84.728848,195.259813,118.377835,162.798667,195.209250


### 2) Evaporative Demand
Reference ET (ET0) Monitoring Data set (uses MERRA-2 atmospheric reanalysis)

In [5]:
# Get the extent of ETos data
filn_etos = './data/earthobs/ETos/ETos_fine_1981-2020.nc'
nc = Dataset(filn_etos, 'r')
lat = -np.array(nc.variables['lat']); dy = 0.125
lon = np.array(nc.variables['lon']); dx = 0.125
extent = [lon[0]-dx/2, lon[-1]+dx/2, lat[-1]-dy/2, lat[0]+dy/2]   # [minx, maxx, miny, maxy]

# (1) Create a graticule of ETos data
filn_out = './data/shapefile/SO_admin2_etos.shp'
CreateGridBox_subextent(filn_out, extent, dx, dy, sub_extent, set_print=False)
    
# (2) Intersection of ETos with admin level-2 boundaries
ref = './data/shapefile/SO_admin2_etos.shp'
adm = './data/shapefile/SO_admin2_1990_revised.shp'
IntersectShapefiles(ref, adm, outSHP=ref, set_print=False)

# (3) Calculate the areas of the intersected polygons
# Reproject to Somalia UTM: UTM zone 38N (EPSG:20538)
filn_out = './data/shapefile/SO_admin2_etos.shp'
grid_dist = gpd.read_file('./data/shapefile/SO_admin2_etos.shp')
grid_dist = grid_dist.to_crs(epsg=20538)    # UTM zone 18S
# Caculate areas of all split polygons
grid_dist["area_km2"] = grid_dist['geometry'].area / 10**6
# Save GeoDataFrame to Shapefile
grid_dist.to_file(filn_out)
print('%s is saved.' % filn_out)

./data/shapefile/SO_admin2_etos.shp is saved.


In [6]:
# Load ETos data
etos = np.array(nc.variables['ETos'])
etos = etos[:,::-1,:]         # NetCDF data is flipped
ntim, nlat, nlon = etos.shape
tim = nc.variables['time']
tim = num2date(tim[:], tim.units, tim.calendar)
tim = ['%04d-%02d' % (t.year, t.month) for t in tim]
tim = pd.to_datetime(tim, format='%Y-%m')
tim = tim.to_period('M')
etos = etos.reshape([ntim,nlat*nlon])

# Spatial averages in administrative units
grid = gpd.read_file('./data/shapefile/SO_admin2_etos.shp')
listID = np.sort(grid.FNID.unique())
nID = len(listID)
etos_dist = pd.DataFrame(np.zeros([ntim,nID]), index=tim, columns=listID)
etos_dist.index.name = 'time'
for fnid in listID:
    subID = grid[grid.FNID == fnid].ID.astype(int)
    subData = etos[:, subID.values]
    subData[np.isnan(subData)] = 0    # Assume 0 values at NaN grids
    subArea = grid[grid.FNID == fnid].area_km2
    # Area-averaged data
    etos_dist[fnid] = (subData @ subArea)/subArea.sum()
save_hdf('./data/earthobs/SO_admin2_etos.hdf',etos_dist)
del etos, lat, lon, tim
etos_dist.head()

./data/SO_admin2_etos.hdf is saved.


,SO1990A21101,SO1990A21102,SO1990A21103,SO1990A21104,SO1990A21201,SO1990A21202,SO1990A21203,SO1990A21301,SO1990A21302,SO1990A21303,...,SO1990A22604,SO1990A22605,SO1990A22606,SO1990A22701,SO1990A22702,SO1990A22703,SO1990A22801,SO1990A22802,SO1990A22803,SO1990A22804
time,,,,,,,,,,,,,,,,,,,,,
1981-01,154.923766,149.890819,147.676142,149.988299,155.946775,147.701981,148.496767,166.864062,183.012041,160.385631,...,240.831983,247.965481,262.089501,241.907174,217.081127,251.925101,217.196124,242.588044,211.674128,196.797090
1981-02,149.086542,146.640680,145.980852,147.509156,154.260121,144.120182,145.008143,168.984857,184.765616,161.667371,...,234.585529,239.024149,254.453304,230.711113,204.164114,245.359792,208.272786,233.984943,204.291552,185.790154
1981-03,163.432622,167.723676,164.569557,160.849498,178.966591,176.497229,167.745362,196.170205,217.614981,186.874226,...,220.484666,232.589830,242.134478,197.066908,178.655696,213.846653,184.914221,201.938172,178.938363,167.221511
1981-04,151.783335,169.146789,171.252679,162.524911,162.089230,169.865413,155.987737,172.017217,189.706183,162.591478,...,166.252215,174.682481,177.505661,158.677297,155.762867,164.897558,164.841615,161.143939,160.753003,153.156825
1981-05,182.973390,201.739250,207.001174,198.187711,184.236077,198.790717,181.353521,187.486778,204.948976,178.312332,...,163.038574,172.373040,161.944940,136.989380,131.240546,154.183190,152.282607,151.472816,148.397358,134.974777


### 3) Soil Moisture from FLDAS
[Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System](https://ldas.gsfc.nasa.gov/fldas) </br>
FLDAS data can be downloaded from here: https://hydro1.gesdisc.eosdis.nasa.gov/data/FLDAS/

In [62]:
# Get the extent of FLDAS Africa data
filn_fldas = './data/earthobs/FLDAS/SoilMoi00_10cm_tavg_Africa_1982-2020.nc'
nc = Dataset(filn_fldas, 'r')
lat = -np.array(nc.variables['Y']); dy = 0.1
lon = np.array(nc.variables['X']); dx = 0.1
extent = [lon[0]-dx/2, lon[-1]+dx/2, lat[-1]-dy/2, lat[0]+dy/2]   # [minx, maxx, miny, maxy]

# (1) Create a graticule of FLADS Africa data
filn_out = './data/shapefile/SO_admin2_fldas.shp'
CreateGridBox_subextent(filn_out, extent, dx, dy, sub_extent, set_print=False)
    
# (2) Intersection of FLDAS Africa with admin level-2 boundaries
ref = './data/shapefile/SO_admin2_fldas.shp'
adm = './data/shapefile/SO_admin2_1990_revised.shp'
IntersectShapefiles(ref, adm, outSHP=ref, set_print=False)

# (3) Calculate the areas of the intersected polygons
# Reproject to Somalia UTM: UTM zone 38N (EPSG:20538)
filn_out = './data/shapefile/SO_admin2_fldas.shp'
grid_dist = gpd.read_file('./data/shapefile/SO_admin2_fldas.shp')
grid_dist = grid_dist.to_crs(epsg=20538)    # UTM zone 18S
# Caculate areas of all split polygons
grid_dist["area_km2"] = grid_dist['geometry'].area / 10**6
# Save GeoDataFrame to Shapefile
grid_dist.to_file(filn_out)
print('%s is saved.' % filn_out)

./data/shapefile/SO_admin2_fldas.shp is saved.


In [68]:
# Load FLDAS Africa data
smos = np.array(nc.variables['SoilMoi00_10cm_tavg'])
smos = smos[:,::-1,:]         # NetCDF data is flipped
ntim, nlat, nlon = smos.shape
tim = nc.variables['time']
tim = num2date(tim[:], tim.units, tim.calendar)
tim = ['%04d-%02d' % (t.year, t.month) for t in tim]
tim = pd.to_datetime(tim, format='%Y-%m')
tim = tim.to_period('M')
smos = smos.reshape([ntim,nlat*nlon])

# Spatial averages in administrative units
grid = gpd.read_file('./data/shapefile/SO_admin2_fldas.shp')
listID = np.sort(grid.FNID.unique())
nID = len(listID)
smos_dist = pd.DataFrame(np.zeros([ntim,nID]), index=tim, columns=listID)
smos_dist.index.name = 'time'
for fnid in listID:
    subID = grid[grid.FNID == fnid].ID.astype(int)
    subData = smos[:, subID.values]
    subData[subData == -9999] = 0    # Assume 0 values at -9999 grids
    subArea = grid[grid.FNID == fnid].area_km2
    # Area-averaged data
    smos_dist[fnid] = (subData @ subArea)/subArea.sum()
save_hdf('./data/earthobs/SO_admin2_smos.hdf',smos_dist)
# del smos, lat, lon, tim
smos_dist.head()

./data/earthobs/SO_admin2_smos.hdf is saved.


,SO1990A21101,SO1990A21102,SO1990A21103,SO1990A21104,SO1990A21201,SO1990A21202,SO1990A21203,SO1990A21301,SO1990A21302,SO1990A21303,...,SO1990A22604,SO1990A22605,SO1990A22606,SO1990A22701,SO1990A22702,SO1990A22703,SO1990A22801,SO1990A22802,SO1990A22803,SO1990A22804
time,,,,,,,,,,,,,,,,,,,,,
1982-01,0.190730,0.191138,0.198509,0.198257,0.164483,0.178351,0.175048,0.155879,0.152458,0.157007,...,0.172576,0.149976,0.155376,0.161033,0.152384,0.173635,0.160540,0.170491,0.165737,0.133500
1982-02,0.207438,0.186628,0.187185,0.197895,0.167096,0.163223,0.185242,0.153123,0.147840,0.158329,...,0.152513,0.145949,0.147584,0.147506,0.144322,0.151967,0.145315,0.153484,0.147432,0.128825
1982-03,0.208490,0.173811,0.169408,0.180397,0.173395,0.153494,0.187303,0.151646,0.147217,0.158592,...,0.148254,0.146559,0.150883,0.150105,0.141996,0.147835,0.150651,0.161720,0.146956,0.127367
1982-04,0.286735,0.254847,0.242036,0.245817,0.260019,0.230747,0.278065,0.216879,0.212642,0.228835,...,0.279001,0.267042,0.275015,0.243471,0.217583,0.253451,0.213898,0.248143,0.255511,0.187293
1982-05,0.283194,0.246197,0.210148,0.216771,0.283890,0.262207,0.288170,0.274431,0.263982,0.277819,...,0.262575,0.277512,0.264848,0.319152,0.307312,0.295808,0.282585,0.301040,0.317699,0.257365


### 4) Water Requirement Satisfaction Index data (WRSI based on CHIRPS and ET0)

### 5) NDVI MAX
NASA GIMMS AVHRR Global NDVI3g.v1 (Normalized Difference Vegetation Index-3rd generation version 1 </br>
MODIS data download from here: https://modis.gsfc.nasa.gov/data/dataprod/mod13.php


The latest NDVI data is [AVHRR Normalized Difference Vegetation Index (NDVI) Version 5](https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C01558/html), [(NCEI's FTP server)](https://www.ncei.noaa.gov/data/avhrr-land-normalized-difference-vegetation-index/access/).


In [4]:
# Get the extent of NDVI EastAfrica data
# filn_ndvi = './data/earthobs/NDVI/AVHRR_NDVI_v5/AVHRR-Land_v005-preliminary_AVH13C1_NOAA-19_20200619_c20200620100604.nc'
filn_ndvi = './data/earthobs/NDVI/GIMMS-NDVI-3g.v1/EastAfrica_monthly_ndvi3g_geo_v1_1982-2015.nc4'
nc = Dataset(filn_ndvi, 'r')
lat = np.array(nc.variables['latitude']); dy = 1/12
lon = np.array(nc.variables['longitude']); dx = 1/12
extent = [lon[0]-dx/2, lon[-1]+dx/2, lat[-1]-dy/2, lat[0]+dy/2]   # [minx, maxx, miny, maxy]

# (1) Create a graticule of NDVI EastAfrica data
filn_out = './data/shapefile/SO_admin2_ndvi.shp'
CreateGridBox_subextent(filn_out, extent, dx, dy, sub_extent, set_print=False)

# (2) Intersection of NDVI EastAfrica with admin level-2 boundaries
ref = './data/shapefile/SO_admin2_ndvi.shp'
adm = './data/shapefile/SO_admin2_1990_revised.shp'
IntersectShapefiles(filn_out, adm, outSHP=ref, set_print=False)

# (3) Calculate the areas of the intersected polygons
# Reproject to Somalia UTM: UTM zone 38N (EPSG:20538)
filn_out = './data/shapefile/SO_admin2_ndvi.shp'
grid_dist = gpd.read_file('./data/shapefile/SO_admin2_ndvi.shp')
grid_dist = grid_dist.to_crs(epsg=20538)    # UTM zone 18S
# Caculate areas of all split polygons
grid_dist["area_km2"] = grid_dist['geometry'].area / 10**6
# Save GeoDataFrame to Shapefile
grid_dist.to_file(filn_out)
print('%s is saved.' % filn_out)

./data/shapefile/SO_admin2_ndvi.shp is saved.


In [64]:
# Load NDVI EastAfrica data
ndvi = np.array(nc.variables['ndvi'])
ntim, nlat, nlon = ndvi.shape
tim = nc.variables['time']
tim = num2date(tim[:], tim.units, tim.calendar)
tim = ['%04d-%02d' % (t.year, t.month) for t in tim]
tim = pd.to_datetime(tim, format='%Y-%m')
tim = tim.to_period('M')
ndvi = ndvi.reshape([ntim,nlat*nlon])

# Spatial averages in administrative units
grid = gpd.read_file('./data/shapefile/SO_admin2_ndvi.shp')
listID = np.sort(grid.FNID.unique())
nID = len(listID)
ndvi_dist = pd.DataFrame(np.zeros([ntim,nID]), index=tim, columns=listID)
ndvi_dist.index.name = 'time'
for fnid in listID:
    subID = grid[grid.FNID == fnid].ID.astype(int)
    subData = ndvi[:, subID.values]
    subArea = grid[grid.FNID == fnid].area_km2
    # Missing grids control
    rdx = np.sum((subData == -3000) | (subData > 10000), 0) > 0
    subData = subData[:,~rdx]
    subArea = subArea[~rdx]
    # Area-averaged data
    ndvi_dist[fnid] = (subData @ subArea)/subArea.sum()
ndvi_dist = ndvi_dist/10000 # Scale
save_hdf('./data/earthobs/SO_admin2_ndvi.hdf',ndvi_dist)
del ndvi, lat, lon, tim
ndvi_dist.head()

./data/earthobs/SO_admin2_ndvi.hdf is saved.


,SO1990A21101,SO1990A21102,SO1990A21103,SO1990A21104,SO1990A21201,SO1990A21202,SO1990A21203,SO1990A21301,SO1990A21302,SO1990A21303,...,SO1990A22604,SO1990A22605,SO1990A22606,SO1990A22701,SO1990A22702,SO1990A22703,SO1990A22801,SO1990A22802,SO1990A22803,SO1990A22804
time,,,,,,,,,,,,,,,,,,,,,
1982-01,0.175883,0.144743,0.120989,0.119453,0.153669,0.120087,0.180532,0.168490,0.186043,0.157988,...,0.321234,0.190146,0.211402,0.489976,0.443677,0.386116,0.416460,0.413776,0.452139,0.336220
1982-02,0.175684,0.149730,0.138611,0.128997,0.151979,0.117038,0.179644,0.164523,0.182914,0.155491,...,0.313244,0.197443,0.210345,0.446137,0.414324,0.346140,0.403216,0.370443,0.438677,0.336224
1982-03,0.188743,0.165632,0.166720,0.133900,0.157875,0.122469,0.191647,0.176366,0.195160,0.164067,...,0.313192,0.198470,0.210518,0.409765,0.407700,0.321977,0.400006,0.348298,0.427220,0.346904
1982-04,0.208692,0.162920,0.181682,0.148119,0.155374,0.118772,0.207507,0.164727,0.189526,0.153078,...,0.406325,0.190866,0.231483,0.465389,0.411591,0.416863,0.443154,0.418141,0.462811,0.305589
1982-05,0.235084,0.162819,0.133946,0.131992,0.165829,0.118881,0.231997,0.179619,0.216958,0.163329,...,0.518273,0.250340,0.333505,0.590538,0.569690,0.513229,0.589147,0.528716,0.573515,0.432206
